In [19]:
import numpy as np
import pandas as pd
from ast import literal_eval

# Task 1 Класичний нейрон з довільною кількістю входів

### Режим навчання

In [20]:
frame = pd.read_csv('input_1.csv', sep=',', index_col=0)
data = frame.to_dict()
data

{'Data': {'X': '[2, 4, 7, 3]',
  'Y_expected': '0.7',
  'W': '[0.2, 0.13, 0.27, 0.3]',
  'delta_max': '0.01'}}

In [3]:
x = np.array(literal_eval(data['Data']['X']), dtype=float)
y_expected = float(data['Data']['Y_expected'])
W = np.array(literal_eval(data['Data']['W']), dtype=float)
delta_max = float(data['Data']['delta_max'])

In [4]:
class classifier():
    
    def fit(self, x, y_expected, W, delta_max):

        count = 0
        while True:
            count += 1
            X = np.dot(W.T, x)
            y = 1/(1+np.exp(-X))
            delta = np.abs((y_expected - y)/y_expected)
            if delta < delta_max:
                break
            else:
                delta_y = y*(1-y)*(y_expected - y)
                delta_w = delta_y * x
                W += delta_w
        
        return count, W, y
    
    def predict(self, x, W):
        X = np.dot(W.T, x)
        y = 1/(1+np.exp(-X))
        return y

In [5]:
count, W, y = classifier().fit(x, y_expected, W, delta_max)
accuracy = 100 - np.abs((y_expected - y)/y_expected)*100

In [6]:
frame = pd.DataFrame({'Data': [count, np.array2string(W, precision=7, separator=', '), y_expected, y, str(accuracy)+" %"]},
                     index=['Кількість ітерацій', 'W', 'Y очікуване', 'Y реальне', 'Accuracy'])

In [7]:
frame.to_csv('output_1.csv', sep=',', header=True, index=True)

In [8]:
frame.style.set_properties(subset=['Data'], **{'width': '450px'})

,Data
Кількість ітерацій,237
W,"[ 0.1272668, -0.0154664, 0.0154338, 0.1909002]"
Y очікуване,0.7
Y реальне,0.705454
Accuracy,99.2208976334287 %


### Режим розпізнавання

In [9]:
y = classifier().predict([6, 1, 4, 8], W)
y_expected = 0.7
accuracy = 100 - np.abs((y_expected - y)/y_expected)*100
print("Y реальне: "+str(y))
print("Y очікуване: "+str(y_expected))
print(str(accuracy)+' %')

Y реальне: 0.9119010634317091
Y очікуване: 0.7
69.72841950975584 %


# Task 2 ДШП

### Режим навчання

In [10]:
frame = pd.read_csv('input_2.csv', sep=',', index_col=0)
data = frame.to_dict()

In [11]:
x = np.array(literal_eval(data['Data']['X']), dtype=float)
y_expected = np.array(literal_eval(data['Data']['Y_expected']))
L = int(data['Data']['L'])
W = {}
for i in range(1, L+1):
    W['W'+str(i)] = np.array(literal_eval(data['Data']['W'+str(i)]), dtype=float)
delta_max = float(data['Data']['delta_max'])

In [12]:
class NN_classifier():
    
    def linear_forward(self, A, W):
        return np.dot(W, A)

    def linear_activation_forward(self, A_prev, W):
        Z = self.linear_forward(A_prev, W)
        A = 1/(1+np.exp(-Z))
        return A
    
    def fit(self, x, y_expected, L, W, delta_max):
        count = 0
        y = {'0': x,
             str(L+1): y_expected}
        delta_y = {}
        delta_w = {}
        while True:
            count += 1
            for i in range(L):
                y_prev = y[str(i)]
                y[str(i+1)] = self.linear_activation_forward(y_prev, W['W'+str(i+1)])
            delta = np.abs((y[str(L+1)] - y[str(L)])/y[str(L+1)])
            if delta < delta_max:
                break
            else:
                delta_y[str(L-1)] = y[str(L)]*(1-y[str(L)])*(y[str(L+1)]-y[str(L)])
                assert(delta_y[str(L-1)].shape == y[str(L)].shape)
                delta_w[str(L-1)] = delta_y[str(L-1)]*y[str(L-1)]
                W['W'+str(L)] += delta_w[str(L-1)].T
                assert(W['W'+str(L)].shape == delta_w[str(L-1)].T.shape)
                for i in range(L-1, 0, -1):
                    delta_y[str(i-1)] = y[str(i)]*(1-y[str(i)])*np.dot(W['W'+str(i+1)].T, delta_y[str(i)])
                    assert(delta_y[str(i)].shape == y[str(i+1)].shape)
                    delta_w[str(i-1)] = np.dot(delta_y[str(i-1)], y[str(i-1)].T)
                    W['W'+str(i)] += delta_w[str(i-1)]
                    assert(W['W'+str(i)].shape == delta_w[str(i-1)].shape)

        return count, W, y[str(L)]
    
    def predict(self, x, W, L):
        y = {'0': x}
        for i in range(L):
            y_prev = y[str(i)]
            y[str(i+1)] = self.linear_activation_forward(y_prev, W['W'+str(i+1)])
            
        return y[str(L)]

In [13]:
count, W, y = NN_classifier().fit(x, y_expected, L, W, delta_max)
accuracy = 100 - np.abs((y_expected - y)/y_expected)*100

In [14]:
frame = pd.DataFrame({'Data': [count, np.array2string(W['W1'], precision=7, separator=', '),
                               np.array2string(W['W2'], precision=7, separator=', '),
                               np.array2string(W['W3'], precision=7, separator=', '),
                               np.array2string(W['W4'], precision=7, separator=', '),
                               y_expected[0][0], y[0][0], str(accuracy[0][0])+" %"]},
                               index=['Кількість ітерацій', 'W1', 'W2', 'W3', 'W4', 'Y очікуване', 'Y реальне', 'Accuracy'])

In [15]:
frame.to_csv('output_2.csv', sep=',', header=True, index=True)

In [16]:
frame.style.set_properties(subset=['Data'], **{'width': '450px'})

,Data
Кількість ітерацій,87
W1,"[[0.2030074], [0.1219651]]"
W2,"[[0.1680268, 0.2081557], [0.2384005, 0.1578509], [0.4546843, 0.3243779]]"
W3,"[[ 0.200648 , 0.3708346, 0.1558245], [-0.2395036, 0.2607128, 0.2266751]]"
W4,"[[0.6940249, 0.7381032]]"
Y очікуване,0.7
Y реальне,0.693098
Accuracy,99.01401996344205 %


### Режим розпізнавання

In [18]:
y = NN_classifier().predict([[35]], W, L)
y_expected = 0.7
accuracy = 100 - np.abs((y_expected - y)/y_expected)*100
print("Y реальне: "+str(y[0][0]))
print("Y очікуване: "+str(y_expected))
print(str(accuracy[0][0])+' %')

Y реальне: 0.6949699865664937
Y очікуване: 0.7
99.28142665235625 %
